<h1 align="center">
Limpeza Catálogo de Teses e Dissertações <br/>
<img src="https://dadosabertos.capes.gov.br/img/caixa.png"  alt="Dados Capes"/>
</h1>

Em nosso conjunto de dados possuímos informações sobre teses e dissertações defendidas no período de 1987-2019. No notebook de [download de metadados](https://github.com/AcademicAI/topicos-teses-e-dissertacoes/blob/main/1_download_metadados.ipynb), realizamos o download e junção de todos os três conjuntos.

Desejamos extrair informações e estatísticas sobre o conteúdo das teses e dissertações, para isso utilizamos a coluna `resumo`. 

Com base no resumo, gostaríamos de responder algumas perguntas sobre os dados,tais como:

- quais são os tópicos mais presentes em uma grande área de conhecimento? E em uma área de conhecimento?
- quais são os tópicos mais abordados em cada período de tempo?
- os tópicos desses trabalhos são diferentes em cada região? E em nível de estadual ou nível de universidade?

Entretanto para que isso seja possível, precisamos verificar se não houve erro de preenchimento ou valores faltantes. Assim realizaremos uma limpeza de dados nas colunas referentes a essas informações.



In [1]:
from shutil import copyfile
import pandas as pd
import numpy as np
import re
import os

## Seleção das colunas de interesse

Primeiro carregamos o arquivo contendo com conjunto de dados, em seguida selecionamos as colunas que são necessárias para responder nossas perguntas.

In [2]:
#@title Caminho do arquivo
#@markdown Inclua o caminho do arquivo onde você salvou.
filepath = "/content/drive/MyDrive/AcademicAI/catalogo-de-teses-e-dissertacoes/catalogo-de-teses-e-dissertacoes.csv" #@param {type:"string"}

# copia arquivo para o diretório do colab
copyfile(filepath, '/content/catalogo-de-teses-e-dissertacoes.csv')

'/content/catalogo-de-teses-e-dissertacoes.csv'

In [3]:
data = pd.read_csv('catalogo-de-teses-e-dissertacoes.csv')
data.info()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235796 entries, 0 to 1235795
Data columns (total 22 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ano_base                1235796 non-null  float64
 1   titulo_producao         1235734 non-null  object 
 2   resumo                  1235571 non-null  object 
 3   palavra_chave           1234724 non-null  object 
 4   idioma                  1165930 non-null  object 
 5   grau_academico          1235795 non-null  object 
 6   cod_grande_area         1235795 non-null  float64
 7   nome_grande_area        1235795 non-null  object 
 8   cod_area_conhecimento   1235795 non-null  float64
 9   nome_area_conhecimento  1235795 non-null  object 
 10  cod_programa            1235796 non-null  object 
 11  nome_programa           1235795 non-null  object 
 12  sigla_entidade_ensino   1235795 non-null  object 
 13  nome_entidade_ensino    1235795 non-null  object 
 14  no

Dentre as colunas de interesse selecionamos:

- ano_base
- resumo
- nome_grande_area
- nome_area_conhecimento
- regiao
- sigla_uf
- sigla_entidade_ensino

In [4]:
data = data[['ano_base', 'resumo', 'nome_grande_area', 'nome_area_conhecimento', 'regiao', 'sigla_uf', 'sigla_entidade_ensino']]
data.head()

,ano_base,resumo,nome_grande_area,nome_area_conhecimento,regiao,sigla_uf,sigla_entidade_ensino
0,1987.0,"EM 1985, REALIZOU-SE UMA PESQUISA EM TRES UNID...",CIÊNCIAS HUMANAS,EDUCAÇÃO,SUDESTE,SP,PUC/SP
1,1987.0,ESTUDA A TRAJETORIA DE FORMACAO DO CONTEXTO IN...,CIÊNCIAS SOCIAIS APLICADAS,ADMINISTRAÇÃO PÚBLICA,NORDESTE,BA,UFBA
2,1987.0,A RECEPCAO DA TELEVISAO PELOS PEQUENOS PRODUTO...,CIÊNCIAS SOCIAIS APLICADAS,ADMINISTRAÇÃO PÚBLICA,NORDESTE,PE,UFRPE
3,1987.0,ESTUDO DAS CONSEQUENCIAS JURIDICAS DA SEPARACA...,CIÊNCIAS SOCIAIS APLICADAS,DIREITO,SUDESTE,SP,USP
4,1987.0,O PODER DISCRICIONARIO ESTA PRESENTE NA ATIVID...,CIÊNCIAS SOCIAIS APLICADAS,DIREITO,SUDESTE,SP,USP


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235796 entries, 0 to 1235795
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ano_base                1235796 non-null  float64
 1   resumo                  1235571 non-null  object 
 2   nome_grande_area        1235795 non-null  object 
 3   nome_area_conhecimento  1235795 non-null  object 
 4   regiao                  1235795 non-null  object 
 5   sigla_uf                1235795 non-null  object 
 6   sigla_entidade_ensino   1235795 non-null  object 
dtypes: float64(1), object(6)
memory usage: 66.0+ MB


Podemos perceber que há palavras não acentuadas nos textos de 1987. Também notamos que há valores `nan` na coluna de resumo, como a nossa análise a considera, realizamos a remoção das linhas onde o `resumo` não foi cadastrado.

In [6]:
data = data.dropna(subset=['resumo'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1235571 entries, 0 to 1235795
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ano_base                1235571 non-null  float64
 1   resumo                  1235571 non-null  object 
 2   nome_grande_area        1235571 non-null  object 
 3   nome_area_conhecimento  1235571 non-null  object 
 4   regiao                  1235571 non-null  object 
 5   sigla_uf                1235571 non-null  object 
 6   sigla_entidade_ensino   1235571 non-null  object 
dtypes: float64(1), object(6)
memory usage: 75.4+ MB


Por fim removemos as linhas duplicadas cadastradas em nosso *dataset*.



In [7]:
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1226895 entries, 0 to 1235795
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ano_base                1226895 non-null  float64
 1   resumo                  1226895 non-null  object 
 2   nome_grande_area        1226895 non-null  object 
 3   nome_area_conhecimento  1226895 non-null  object 
 4   regiao                  1226895 non-null  object 
 5   sigla_uf                1226895 non-null  object 
 6   sigla_entidade_ensino   1226895 non-null  object 
dtypes: float64(1), object(6)
memory usage: 74.9+ MB


## Limpeza do Resumo

Para a limpeza do resumo começamos removendo os resumos com o texto `NAO INFORMADO.`.

In [8]:
data = data[~data['resumo'].str.contains('NAO INFORMADO.', regex=False)]

Sabemos que em nosso dataset há textos sem valor para nossa análise, como exemplo o texto `NAO INFORMADO`. 

Na linha abaixo buscamos os resumos com até 60 caracteres.

In [ ]:
for index, _ in data[data['resumo'].str.len() <= 60]['resumo'].str.lower().value_counts().items():
  print(index)

Podemos notar que a maioria das sentenças com até 60 caracteres não refletem informação útil sobre a tese/dissertação, optamos por removê-las.

In [10]:
data = data[data['resumo'].str.len() > 60] 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1220066 entries, 0 to 1235795
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ano_base                1220066 non-null  float64
 1   resumo                  1220066 non-null  object 
 2   nome_grande_area        1220066 non-null  object 
 3   nome_area_conhecimento  1220066 non-null  object 
 4   regiao                  1220066 non-null  object 
 5   sigla_uf                1220066 non-null  object 
 6   sigla_entidade_ensino   1220066 non-null  object 
dtypes: float64(1), object(6)
memory usage: 74.5+ MB


Fazemos mais uma checagem para trabalhos com até 100 caracteres a fim de descobrir padrões que podemos usar para remover as linhas inconsistentes, se possível.

In [ ]:
for index, _ in data[data['resumo'].str.len() < 100]['resumo'].str.lower().value_counts().items():
  print(index)

Nessa amostra de até 100 caracteres, percebemos alguns padrões:
1. sequência de caracteres repetidos
2. Textos com url

Em 1 podemos remover a repetição dos caracteres.
Em 2 podemos remover as URLs.

Por fim, excluímos os resumos com até 50 caracteres.

In [12]:
# substitui sequências como aaaaaaaaaaaaaaaaaaaaa por a, ou ------ por -
resumo_sem_repetidos = data['resumo'].str.replace(r'([\w.*-])\1{2,}', r'\1')

In [13]:
# primeiro replace: substitui sequências como NONONONO por NO
# segundo replace: remove hiperlinks
resumo_sem_repetidos = resumo_sem_repetidos.str.replace(r'([\w.*-][\w.*-])\1{2,}', r'\1').str.replace(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', ' ', flags=re.IGNORECASE)

In [14]:
# resumos com menos de 50 caracteres
for index in data[resumo_sem_repetidos.str.len() < 50]['resumo'].values:
  print(index)

O EXPERIMENTO FOI CONDUZIDO DE DEZ.DE 83 A JULHO 87 FORAM ESTUDADAS   TRES FREQUENCIAS E DUAS ALTURAS DE CORTE NA PRODUTIVIDADE E FISIOLOGIADE PERFILHAMENTO DO CAPIM SETARIA  EM DELINEAMENTO DE BLOCOS AO ACASO COM  4 REPETICOES  E UM ARRANJO FA-TORIAL 3X2 NOS TRATAMENTOS FREQUEN-CIA  DE CORTE  INFLUIU SIGNIFICATI-VAMENTE NA ELEVACAO DO MERISTEMA   APICAL NO NUMERO DE PERFILHOS BASI-LARES E LATERAIS HOUVERAM EFEITOS SIGNIFICATIVOS DE DATAS DE CORTE SOBRE   TODAS AS VARIAVEIS ESTUDADAS ALTURAS DE CORTE INFLUENCIARAM SIG-NIFICATIVAMENTE A PRODUCAO DE MATE-RIA SECA O ELEVACAO DO MERISTEMA A-PICAL AS CONDICOES CLIMATICAS TIVE- SOBRE ESSE TIPO DE PERFILHO.
ANTECEDENTES E CAUSAS. A DECADA DE 20 NO BRASIL.PARAIBA E PERNAMBUCO  EM 1930. IDEOLOGIA DA REVOLUCAO DE 1930.O GOVERNO CARLOS DE LIMA CAVALANTI. CONTROVERSIAS E REALIZACOES. ADOS BIOGRAFICOS. CARLOS DE LIMA E OS DIFERENTES SEGMENTOS SOCIAIS  EM PERNAMBUCO.O AMBIENTE AS VESPE- RAS DA REVOLTA E SUA CRONOLOGIA.   FRACASSO E REACAO OFICIAL. R

Acima notamos que a maioria dos resumos formados com caracteres repetidos ou links em sua maioria não continham informações relevantes para nossa análise. Desse modo, optamos por deixar apenas resumos com mais de 50 caracteres.

In [15]:
data = data[resumo_sem_repetidos.str.len() > 50]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1219950 entries, 0 to 1235795
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ano_base                1219950 non-null  float64
 1   resumo                  1219950 non-null  object 
 2   nome_grande_area        1219950 non-null  object 
 3   nome_area_conhecimento  1219950 non-null  object 
 4   regiao                  1219950 non-null  object 
 5   sigla_uf                1219950 non-null  object 
 6   sigla_entidade_ensino   1219950 non-null  object 
dtypes: float64(1), object(6)
memory usage: 74.5+ MB


Por fim, removemos as linhas onde o resumo se repete.



In [16]:
data = data.drop_duplicates(subset=['resumo'])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1219170 entries, 0 to 1235795
Data columns (total 7 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ano_base                1219170 non-null  float64
 1   resumo                  1219170 non-null  object 
 2   nome_grande_area        1219170 non-null  object 
 3   nome_area_conhecimento  1219170 non-null  object 
 4   regiao                  1219170 non-null  object 
 5   sigla_uf                1219170 non-null  object 
 6   sigla_entidade_ensino   1219170 non-null  object 
dtypes: float64(1), object(6)
memory usage: 74.4+ MB


## Limpeza nas demais colunas

Primeiro verificamos se há inconsistência nas demais colunas do dataset.

In [17]:
# Ano
data['ano_base'].unique()

array([1987., 1988., 1989., 1990., 1991., 1992., 1993., 1994., 1995.,
       1996., 1997., 1998., 1999., 2000., 2001., 2002., 2003., 2004.,
       2005., 2006., 2007., 2008., 2009., 2010., 2011., 2012., 2013.,
       2014., 2015., 2016., 2017., 2018., 2019.])

In [18]:
# nome_grande_area
data['nome_grande_area'].unique()

array(['CIÊNCIAS HUMANAS', 'CIÊNCIAS SOCIAIS APLICADAS',
       'CIÊNCIAS AGRÁRIAS', 'LINGÜÍSTICA, LETRAS E ARTES',
       'CIÊNCIAS DA SAÚDE', 'CIÊNCIAS EXATAS E DA TERRA',
       'CIÊNCIAS BIOLÓGICAS', 'ENGENHARIAS', 'MULTIDISCIPLINAR'],
      dtype=object)

In [19]:
# nome_grande_area
data['nome_area_conhecimento'].unique()

array(['EDUCAÇÃO', 'ADMINISTRAÇÃO PÚBLICA', 'DIREITO',
       'CIÊNCIAS CONTÁBEIS', 'ADMINISTRAÇÃO', 'ADMINISTRAÇÃO DE EMPRESAS',
       'ADMINISTRAÇÃO DE RECURSOS HUMANOS', 'TEORIA DO DIREITO',
       'DIREITO PÚBLICO', 'CIÊNCIA E TECNOLOGIA DE ALIMENTOS',
       'MEDICINA VETERINÁRIA', 'ANATOMIA PATOLÓGICA ANIMAL',
       'CIÊNCIA DE ALIMENTOS', 'TECNOLOGIA DE ALIMENTOS',
       'ENGENHARIA DE ALIMENTOS', 'MEDICINA VETERINÁRIA PREVENTIVA',
       'CLÍNICA VETERINÁRIA', 'ARQUITETURA E URBANISMO',
       'BIBLIOTECONOMIA', 'COMUNICAÇÃO', 'PLANEJAMENTO URBANO E REGIONAL',
       'FUNDAMENTOS DO PLANEJAMENTO URBANO E REGIONAL', 'DEMOGRAFIA',
       'ECONOMIA AGRÁRIA', 'ECONOMIA', 'CIÊNCIA DA INFORMAÇÃO',
       'SERVIÇO SOCIAL', 'FUNDAMENTOS DO SERVIÇO SOCIAL',
       'ECONOMIAS AGRÁRIA E DOS RECURSOS NATURAIS',
       'ECONOMIA DOS RECURSOS HUMANOS',
       'CRESCIMENTO, FLUTUAÇÕES E PLANEJAMENTO ECONÔMICO',
       'EDUCAÇÃO ESPECIAL', 'TRATAMENTO E PREVENÇÃO PSICOLÓGICA',
       'PSICO

In [20]:
# regiao
data['regiao'].unique()

array(['SUDESTE', 'NORDESTE', 'CENTRO-OESTE', 'SUL', 'NORTE'],
      dtype=object)

In [21]:
# sigla_uf
data['sigla_uf'].unique()

array(['SP', 'BA', 'PE', 'RJ', 'MG', 'DF', 'RS', 'PB', 'RN', 'SC', 'CE',
       'PR', 'AM', 'GO', 'PA', 'ES', 'SE', 'MS', 'AL', 'MT', 'PI', 'MA',
       'RO', 'AC', 'TO', 'RR', 'AP'], dtype=object)

In [22]:
# sigla_uf
data['sigla_entidade_ensino'].unique()

array(['PUC/SP', 'UFBA', 'UFRPE', 'USP', 'UFPE', 'FGV/RJ', 'UFLA',
       'FGV/SP', 'UMESP', 'UNB', 'UFRGS', 'UFMG', 'PUC-RIO', 'UFRJ',
       'UFPB-JP', 'UFRN', 'UFSC', 'UFC', 'UFPR', 'UGF', 'UFV', 'UFSM',
       'UFPEL', 'UFF', 'UFRRJ', 'UNICAMP', 'UEL', 'USP/ESALQ',
       'UNESP/BOT', 'USP/SC', 'PUCCAMP', 'UFPB/C.G.', 'UERJ', 'UFSCAR',
       'UNIFESP', 'USP/RP', 'FCMSCSP', 'UNESP/ARAR', 'UNESP/ARAÇ',
       'USP/FOB', 'PUC/RS', 'UNICAMP/PI', 'UPE', 'FIOCRUZ', 'UNESP/JAB',
       'UFPB/AREIA', 'UNIRIO', 'INPA', 'UNESP-SJRP', 'UNESP/ASS', 'UFG',
       'UNESP-RC', 'UFPA', 'UFJF', 'UNIMEP', 'UFES', 'UFAM', 'INPE',
       'UFOP', 'FURG', 'UEM', 'IME', 'IMPA', 'CBPF', 'ON', 'IFT/UNESP',
       'ITA', 'UFU', 'FEI', 'UNIFEI', 'UNESP/GUAR', 'FUC', 'CBM', 'FUFSE',
       'UCP/RJ', 'CUSC', 'UNESP/FR', 'EST', 'UFTM', 'UTFPR', 'UFCSPA',
       'UFRA', 'UNESP/SJC', 'IUPERJ', 'UNAERP', 'UFERSA', 'USP/CENA',
       'FAENQUIL', 'IAMSPE', 'UNESP-PP', 'UFMS', 'UNESP/MAR', 'UFAL',
       'UNISINOS',

## Salvando dados
Como não encontramos inconsistências nas demais colunas finalizamos a nossa análise e portanto salvamos o novo conjunto de dados.

In [23]:
#@title Pasta do Google Drive 
#@markdown Inclua o caminho da pasta onde você deseja salvar os dados.
destination = "/content/drive/MyDrive/AcademicAI/catalogo-de-teses-e-dissertacoes/" #@param {type:"string"}

data.to_csv(f'{os.path.join(destination, "teses-dissertacoes-cleaned.csv")}', index=False)
